In [ ]:
import numpy as np
import pandas as pd
import tifffile as tf
import matplotlib.pyplot as plt
from pathlib import Path
from map_spots_to_masks import keep_dots_in_cells, keep_dots_parallel

# Map Spots to Cells

In [ ]:
#user
user = "user name"
#name of experiment directory
exp_dir = "exp dir name"
#define channels used
channel = [1,2,3]
#define total number of pos
pos_tot = 100
#define number of z's
num_z = 6

if num_z > 1:
    for c in channel:
        for z in range(num_z):
            spots = []
            mask_paths = []
            for i in range(pos_tot):
                spots.append(f"/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/dots_detected/Channel_{c}/Pos{i}/locations_z_{z}.csv")
                mask_paths.append(f'/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/edges_deleted/MMStack_Pos{i}_z{z}.tif')
            keep_dots_parallel(mask_paths,spots)
else:
    spots = []
    mask_paths = []
    for c in channel:
        for i in range(pos_tot):
            spots.append(f"/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/dots_detected/Channel_{c}/Pos{i}/locations_z_0.csv")
            mask_paths.append(f'/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/edges_deleted/MMStack_Pos{i}_z0.tif')
        keep_dots_parallel(mask_paths,spots)

# If encoding across channels use this for combining spots across channels

In [ ]:
from tqdm import tqdm
#for each channel in a specific position, grab the locations file
#then, concatenate those files and write it out
for i in tqdm(range(pos_tot)):
    for z in range(num_z):
        df_list = []
        for c in channel:
            paths = f"/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/dots_detected/Channel_{c}/spots_in_cells/Pos{i}/locations_z_{z}.csv"
            try:
                df = pd.read_csv(paths)
            except:
                continue
            df_list.append(df)
        try:
            df_comb = pd.concat(df_list).reset_index(drop=True)
        except:
            continue
        output= Path(f"/groups/CaiLab/personal/{user}/raw/{exp_dir}/pyfish_tools/output/dots_detected/Channel_All/Pos{i}")
        output.mkdir(parents=True,exist_ok=True)
        output = output / f"locations_z_{z}.csv"
        df_comb.to_csv(str(output))